# BERT

In [1]:
import os

# Force single-GPU mode in Jupyter
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ["WORLD_SIZE"] = "1"

In [2]:
import ast
import numpy as np
import pandas as pd
from scipy.sparse import hstack

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    log_loss
)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")

import torch
from torch.utils.data import Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    set_seed,
)

from evaluation import *
from data_utils import *

/home/jyao/miniconda3/envs/chatbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-26 10:55:00.858996: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-26 10:55:00.859070: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-26 10:55:00.860200: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-26 10:55:00.867552: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow

In [3]:
# Load and prepare data
train_df, test_df, y, class_names = load_and_prepare_data()

# Get train/val/test splits for BERT
X_train_bert, X_val_bert, X_test_bert, y_train_bert, y_val_bert = prepare_text_pipeline(
    train_df, test_df, y
)

# Ensure positional indexing
y_train_bert = np.asarray(y_train_bert, dtype=int)
y_val_bert   = np.asarray(y_val_bert, dtype=int)

print("Example text:\n", X_train_bert[0][:250])
print("Label:", y_train_bert[0])
print("Classes:", class_names)

Example text:
 ["What is a foreign exchange crisis? What are some notable examples?"] [SEP] ["A foreign exchange crisis refers to a situation where a country faces severe shortage of foreign currencies, usually dollars or euros, to pay for imports or service extern
Label: 1
Classes: {0: 'winner_model_a', 1: 'winner_model_b', 2: 'winner_tie'}


In [4]:
# Torch Dataset
class TextClsDataset(Dataset):
    def __init__(self, texts, labels=None):
        self.texts  = list(texts)
        self.labels = None if labels is None else np.asarray(labels, dtype=int)
    def __len__(self): return len(self.texts)
    def __getitem__(self, idx):
        item = {"text": self.texts[idx]}
        if self.labels is not None:
            item["label"] = int(self.labels[idx])
        return item

ds_train = TextClsDataset(X_train_bert, y_train_bert)
ds_val   = TextClsDataset(X_val_bert, y_val_bert)
ds_test  = TextClsDataset(X_test_bert, labels=None)

In [5]:
# Tokenizer & collator
# MODEL_NAME = "bert-base-uncased"
# MODEL_NAME = "FacebookAI/roberta-base"
# MODEL_NAME = "microsoft/deberta-v3-base"
MODEL_NAME = "microsoft/deberta-v3-large"
max_len = 512
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

def tokenize_fn(batch):
    return tokenizer(
        batch["text"] if isinstance(batch, dict) else batch["text"],
        truncation=True,
        max_length=max_len,
        padding=False,  # dynamic padding via collator
    )

collator = DataCollatorWithPadding(tokenizer=tokenizer)

# HF Trainer expects a dict-like dataset; wrap via map-style adapter
class HFAdapter(Dataset):
    def __init__(self, base_ds, with_labels=True):
        self.base_ds = base_ds
        self.with_labels = with_labels
    def __len__(self): return len(self.base_ds)
    def __getitem__(self, idx):
        ex = self.base_ds[idx]
        toks = tokenizer(
            ex["text"],
            truncation=True,
            max_length=max_len,
            padding=False
        )
        if self.with_labels:
            toks["labels"] = int(ex["label"])
        return toks

hf_train = HFAdapter(ds_train, with_labels=True)
hf_val   = HFAdapter(ds_val,   with_labels=True)
hf_test  = HFAdapter(ds_test,  with_labels=False)

In [6]:
# Model
import os, torch, inspect
id2label = {0: class_names[0], 1: class_names[1], 2: class_names[2]}
label2id = {v: k for k, v in id2label.items()}
num_labels = 3

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"

from collections import Counter
class_counts = Counter(y_train_bert)
total = sum(class_counts.values())

class_weights = [total / class_counts[i] for i in range(len(class_counts))]
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
print("Aggressive class weights:", class_weights)

from torch.nn import CrossEntropyLoss
class WeightedRobertaForSequenceClassification(torch.nn.Module):
    def __init__(self, model_name, class_weights, num_labels=3):
        super().__init__()
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )
        self.class_weights = class_weights
        self.loss_fct = CrossEntropyLoss(weight=self.class_weights, label_smoothing=0.1)

        # Get valid forward args for the base model
        self._valid_args = set(inspect.signature(self.model.forward).parameters.keys())

    def forward(self, **kwargs):
        labels = kwargs.pop("labels", None)
        model_kwargs = {k: v for k, v in kwargs.items() if k in self._valid_args}
    
        outputs = self.model(**model_kwargs)
        logits = outputs.logits
    
        # Ensure logits are 2D
        if logits.ndim == 1:
            logits = logits.unsqueeze(0)
    
        loss = None
        if labels is not None:
            loss = self.loss_fct(logits, labels)
        return type(outputs)(loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions)

# Initialize
model_bert = WeightedRobertaForSequenceClassification(MODEL_NAME, class_weights).to(device)

Aggressive class weights: tensor([2.8647, 2.9248, 3.2360], device='cuda:0')


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# TrainingArguments & Trainer

torch.backends.cuda.matmul.allow_tf32 = True
os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")

# Helper to build TrainingArguments with version-safe keys
def make_training_args(**base_kwargs):
    sig = inspect.signature(TrainingArguments.__init__)
    params = set(sig.parameters.keys())

    # map possibly-renamed keys
    if "evaluation_strategy" in base_kwargs and "evaluation_strategy" not in params and "eval_strategy" in params:
        base_kwargs["eval_strategy"] = base_kwargs.pop("evaluation_strategy")
    if "logging_strategy" in base_kwargs and "logging_strategy" not in params and "log_strategy" in params:
        base_kwargs["log_strategy"] = base_kwargs.pop("logging_strategy")

    # drop keys not supported by this version (e.g., tf32 on older versions)
    safe_kwargs = {k: v for k, v in base_kwargs.items() if k in params}
    return TrainingArguments(**safe_kwargs)

train_args = make_training_args(
    output_dir="runs/bert_cls",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_ratio=0.06,

    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=False,
    save_total_limit=2,

    bf16=True,
    tf32=True,
    dataloader_num_workers=4,
    logging_strategy="epoch",
    log_level="info",
    report_to="none",
)


# Patch the model's loss function
def weighted_loss(outputs, labels):
    loss_fct = CrossEntropyLoss(weight=class_weights)
    return loss_fct(outputs.logits, labels)

from transformers import TrainerCallback

class EpochLossLogger(TrainerCallback):
    def on_epoch_end(self, args, state, control, logs=None, **kwargs):
        train_loss = logs.get("loss") if logs else None
        eval_loss = logs.get("eval_loss") if logs and "eval_loss" in logs else None
        epoch = int(state.epoch)

        train_loss_str = f"{train_loss:.6f}" if train_loss is not None else "N/A"
        eval_loss_str = f"{eval_loss:.6f}" if eval_loss is not None else "N/A"

        print(f"{epoch+1}\t{train_loss_str}\t{eval_loss_str}")
        
trainer = Trainer(
    model=model_bert,
    args=train_args,
    train_dataset=hf_train,
    eval_dataset=hf_val,
    tokenizer=tokenizer,
    data_collator=collator,
    callbacks=[EpochLossLogger()]
)

trainer.train()

/tmp/ipykernel_3750463/3937555970.py:63: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Using auto half precision backend
***** Running training *****
  Num examples = 45,981
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Training with DataParallel so batch size has been adjusted to: 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 4
  Total optimization steps = 2,157
  Number of trainable parameters = 435,064,835
/home/jyao/miniconda3/envs/chatbot/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,8.785200,No log
2,8.803500,No log
3,8.795600,No log



***** Running Evaluation *****
  Num examples = 11496
  Batch size = 16


2	N/A	N/A


Saving model checkpoint to runs/bert_cls/checkpoint-719
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in runs/bert_cls/checkpoint-719/tokenizer_config.json
Special tokens file saved in runs/bert_cls/checkpoint-719/special_tokens_map.json
added tokens file saved in runs/bert_cls/checkpoint-719/added_tokens.json
Deleting older checkpoint [runs/bert_cls/checkpoint-1438] due to args.save_total_limit
/home/jyao/miniconda3/envs/chatbot/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(

***** Running Evaluation *****
  Num examples = 11496
  Batch size = 16


3	N/A	N/A


Saving model checkpoint to runs/bert_cls/checkpoint-1438
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in runs/bert_cls/checkpoint-1438/tokenizer_config.json
Special tokens file saved in runs/bert_cls/checkpoint-1438/special_tokens_map.json
added tokens file saved in runs/bert_cls/checkpoint-1438/added_tokens.json
Deleting older checkpoint [runs/bert_cls/checkpoint-2157] due to args.save_total_limit
/home/jyao/miniconda3/envs/chatbot/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(

***** Running Evaluation *****
  Num examples = 11496
  Batch size = 16


4	N/A	N/A


Saving model checkpoint to runs/bert_cls/checkpoint-2157
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in runs/bert_cls/checkpoint-2157/tokenizer_config.json
Special tokens file saved in runs/bert_cls/checkpoint-2157/special_tokens_map.json
added tokens file saved in runs/bert_cls/checkpoint-2157/added_tokens.json
Deleting older checkpoint [runs/bert_cls/checkpoint-719] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2157, training_loss=8.794741231310848, metrics={'train_runtime': 5257.786, 'train_samples_per_second': 26.236, 'train_steps_per_second': 0.41, 'total_flos': 0.0, 'train_loss': 8.794741231310848, 'epoch': 3.0})

In [8]:
# Predictions (BERT)
pred_output = trainer.predict(hf_val)
pred_logits = pred_output.predictions

# Retrieve correct number of labels
num_labels = trainer.model.model.config.num_labels

# --- Ensure pred_logits is 2D ---
if isinstance(pred_logits, (list, tuple)):
    safe_logits = []
    for p in pred_logits:
        arr = np.array(p)
        if arr.ndim == 2 and arr.shape[1] == num_labels:
            safe_logits.append(arr)
        elif arr.ndim == 1 and arr.size == num_labels:
            safe_logits.append(arr.reshape(1, -1))
    pred_logits = np.vstack(safe_logits)
elif isinstance(pred_logits, torch.Tensor):
    pred_logits = pred_logits.cpu().numpy()

# Convert to probabilities and predictions
y_proba_val_bert = torch.softmax(torch.from_numpy(pred_logits), dim=1).numpy()
y_pred_val_bert  = y_proba_val_bert.argmax(axis=1)

# --- Test predictions ---
pred_output = trainer.predict(hf_test)
pred_logits = pred_output.predictions

if isinstance(pred_logits, (list, tuple)):
    safe_logits = []
    for p in pred_logits:
        arr = np.array(p)
        if arr.ndim == 2 and arr.shape[1] == num_labels:
            safe_logits.append(arr)
        elif arr.ndim == 1 and arr.size == num_labels:
            safe_logits.append(arr.reshape(1, -1))
    pred_logits = np.vstack(safe_logits)
elif isinstance(pred_logits, torch.Tensor):
    pred_logits = pred_logits.cpu().numpy()

y_proba_test_bert = torch.softmax(torch.from_numpy(pred_logits), dim=1).numpy()
y_pred_test_bert  = y_proba_test_bert.argmax(axis=1)
# Compute softmax and predictions
pred_output = trainer.predict(hf_test)
pred_logits = pred_output.predictions

# Ensure all arrays are 2D
if isinstance(pred_logits, (list, tuple)):
    safe_logits = []
    for p in pred_logits:
        arr = np.array(p)
        # If logits are 1D, treat as single sample
        if arr.ndim == 1:
            if arr.size == num_labels:  # single sample logits
                arr = arr.reshape(1, -1)
            else:  # flattened batch? skip or reshape carefully
                arr = arr.reshape(-1, num_labels)
        safe_logits.append(arr)
    pred_logits = np.vstack(safe_logits)
elif isinstance(pred_logits, torch.Tensor):
    pred_logits = pred_logits.cpu().numpy()

# Softmax and argmax
y_proba_test_bert = torch.softmax(torch.from_numpy(pred_logits), dim=1).numpy()
y_pred_test_bert  = y_proba_test_bert.argmax(axis=1)


***** Running Prediction *****
  Num examples = 11496
  Batch size = 16
/home/jyao/miniconda3/envs/chatbot/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



***** Running Prediction *****
  Num examples = 3
  Batch size = 16



***** Running Prediction *****
  Num examples = 3
  Batch size = 16


In [9]:
print("\n================ BERT EVALUATION ================\n")
# Metrics
_ = eval_metrics(y_val_bert, y_pred_val_bert)
eval_classification_report(y_val_bert, y_pred_val_bert, class_names)
# ROC-AUC
_ = eval_roc_auc(y_val_bert, y_proba_val_bert)
# Log-loss
_ = eval_log_loss(y_val_bert, y_proba_val_bert)
_ = eval_log_loss_per_class(y_val_bert, y_proba_val_bert)


================ BERT EVALUATION ================

*** GLOBAL METRICS ***
Accuracy (Global)      : 0.3491
Precision (Macro Avg)  : 0.1164
Recall (Macro Avg)     : 0.3333
F1-Score (Macro Avg)   : 0.1725

*** PER-CLASS EVALUATION ***
Class                Precision    Recall  F1-Score   Support
------------------------------------------------------------
winner_model_a            0.35      1.00      0.52      4013
winner_model_b            0.00      0.00      0.00      3931
winner_tie                0.00      0.00      0.00      3552
------------------------------------------------------------
Macro Avg                 0.12      0.33      0.17     34488
Weighted Avg              0.12      0.35      0.18     34488

*** ROC-AUC EVALUATION ***
ROC-AUC (OvR) : 0.4937

*** LOG-LOSS EVALUATION ***
Log-loss      : 1.0987

*** LOG-LOSS PER CLASS ***
Class 0: 1.0951  (n=4013)
Class 1: 1.1053  (n=3931)
Class 2: 1.0955  (n=3552)


In [10]:
# Confusion Matrix + Plot
cm_bert = eval_confusion_matrix(y_val_bert, y_pred_val_bert, n_classes=y_proba_val_bert.shape[1])
plot_confusion_matrix(cm_bert, class_names, title="Confusion Matrix — DEBERT_L", save_path="results/confusion_matrix/confusion_matrix_debert_l.png")


Confusion Matrix (rows=true, cols=pred):
 [[4013    0    0]
 [3931    0    0]
 [3552    0    0]]
Saved plot to: images/confusion_matrix/confusion_matrix_debert_l.png


In [11]:
# ROC Curves
plot_roc_curves(y_val_bert, y_proba_val_bert, class_names, title_prefix="DEBERT_L ROC", save_path="results/roc/roc_debert_l.png")

Saved plot to: images/roc/roc_debert_l.png


In [12]:
save_roc_to_csv(y_val_bert, y_proba_val_bert, "DEBERT_L", fold_idx=1)

Saved ROC data for class 0 (AUC=0.4888) → results/roc/DEBERT_L_fold1_class0.csv
Saved ROC data for class 1 (AUC=0.4904) → results/roc/DEBERT_L_fold1_class1.csv
Saved ROC data for class 2 (AUC=0.5020) → results/roc/DEBERT_L_fold1_class2.csv


In [13]:
submission_bert = build_submission(
    test_df=test_df,
    y_pred_test=y_pred_test_bert,
    y_proba_test=y_proba_test_bert,
    model_name="debert_l"
)


Saved: results/submission/submission_debert_l.csv
